## This module extracts audio features using OpenSmile

emobase.conf is the configuration file used to extract features for both framewise and for entire audio 

Import necessary Packages

In [1]:
import os
import sys
import glob
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from os import walk
from os.path import splitext
from os.path import join


Using TensorFlow backend.


### Store path for videos from bag of lies dataset

In [86]:
foodir = r'/home/adrikamukherjee/data/BagOfLies/Finalised/'
videolist = list()
count_bagoflies=0
for root, dirs, files in walk(foodir):
  for f in files:
    if splitext(f)[1].lower() == ".mp4":
      videolist.append(join(root, f))
      count_bagoflies +=1
print("number of videos from bag of lies")
print(count_bagoflies)

number of videos from bag of lies
325


In [ ]:
foodir = r'/home/adrikamukherjee/Data_available/Youtube/splitsteal'
count_youtube=0
for root, dirs, files in walk(foodir):
  for f in files:
    if splitext(f)[1].lower() == ".mp4":
      videolist.append(join(root, f))
      count_youtube +=1
print("number of videos from splitsteal youtube")
print(count_youtube)

### Store path for videos youtube data accmulated from 6people, fallon, splitsteal shows

In [2]:
videolist=[]
foodir = r'/home/adrikamukherjee/Youtube/6people'
count_youtube=0
for root, dirs, files in walk(foodir):
  for f in files:
    if splitext(f)[1].lower() == ".mp4":
      videolist.append(join(root, f))
      count_youtube +=1
print("number of videos from 6people youtube")
print(count_youtube)
foodir = r'/home/adrikamukherjee/Youtube/fallon'
count_youtube=0
for root, dirs, files in walk(foodir):
  for f in files:
    if splitext(f)[1].lower() == ".mp4":
      videolist.append(join(root, f))
      count_youtube +=1
print("number of videos from fallon youtube")
print(count_youtube)

number of videos from 6people youtube
146
number of videos from fallon youtube
18


### Store path for videos  from Trial data

In [88]:
#Execute from commandline
#dir_list contains path of all files which contains the audio 
dir_list=["/home/adrikamukherjee/Data_available/Real-life_Deception_Detection_2016/Real-life_Deception_Detection_2016/Clips/Deceptive"
          ,"/home/adrikamukherjee/Data_available/Real-life_Deception_Detection_2016/Real-life_Deception_Detection_2016/Clips/Truthful"]
count_trial=0
for dir in dir_list:
    for filename in glob.glob(os.path.join(dir, '*.mp4')):
        videolist.append(filename)
        count_trial +=1
print("number of videos from trial data")
print(count_trial)

number of videos from trial data
121


In [89]:
len(videolist)

658

### Convert all .mp4 files from the above datasets into .wav files

In [ ]:
#dictionary of input and output file path
dict_input_output={}
output_filename_list=list()
wav_bagoflies=0
wav_youtube=0
wav_trial=0
for filename in videolist:
    output_filename=""
    cmd="/home/adrikamukherjee/ffmpeg-git-20200107-amd64-static/ffmpeg -i " + "\"" + filename+ "\"" + " "
    if "BagOfLies" in filename:
        x=os.path.split(filename)[0]
        x1=os.path.split(filename)[-1]
        y=os.path.split(x)[0]
        y1=os.path.split(x)[-1]
        z1=os.path.split(y)[-1]
        output_filename="Audio_BagOfLies_"+z1+"_"+y1+"_"+x1
        wav_bagoflies +=1
    if "Youtube" in filename:
        x=os.path.split(filename)[0]
        x1=os.path.split(filename)[-1]
        y=os.path.split(x)[0]
        y1=os.path.split(x)[-1]
        z1=os.path.split(y)[-1]
        output_filename="Audio_Youtube_"+z1+"_"+y1+"_"+x1
        wav_youtube +=1
    if "Real-life_Deception_Detection_2016" in filename:
        x=os.path.split(filename)[0]
        x1=os.path.split(filename)[-1]
        y=os.path.split(x)[0]
        y1=os.path.split(x)[-1]
        output_filename="Audio_reallifedeception_"+x1
        wav_trial +=1
    output_filename=output_filename.replace(".mp4",".wav")
    outpath = "/home/adrikamukherjee/wavfiles/"+output_filename
    cmd=cmd+"\""+outpath+"\""
    x=os.system(cmd)
    if(x==0):
        dict_input_output[filename]=outpath
        output_filename_list.append(outpath)
print("converted wav files counts are:")
print("Bagoflies ="+str(wav_bagoflies))
print("youtube = "+str(wav_youtube))
print("Trail = "+str(wav_trial))


In [4]:
count=0
wav_list=[]
path=('/home/adrikamukherjee/wavfiles/')
for files in glob.glob(os.path.join(path, '*.wav')):
    wav_list.append(files)
    count +=1
count

164

In [95]:
dir="/home/adrikamukherjee/Data_available/Audio_features/arff_files_full_video"
for filename in glob.glob(os.path.join(dir, '*.arff')):
    os.remove(filename)

657


In [116]:
dir="/home/adrikamukherjee/Data_available/BagOfLies1/Finalised"
for filename in glob.glob(os.path.join(dir, '*.arff')):
    os.remove(filename)

### Annotate wav files and store in Annotation_audio_features.csv

In [5]:
head=[]
head.append("Path_for_mp4_video")
head.append("Path_for_wav_file")
head.append("csv_file_name")
head.append("csv_file_name_path_fullvideo")
head.append("csv_file_name_path_perframe")
head.append("label")
dir1="/home/adrikamukherjee/Audio_features/arff_files_frame_wise/"
dir2="/home/adrikamukherjee/Audio_features/arff_files_full_video/"

indexi=0
df_input_output=pd.DataFrame(columns = head)
for key,value in dict_input_output.items():
    df_input_output = df_input_output.append(pd.Series(np.nan, index = head),ignore_index=True)
    df_input_output.iloc[indexi, head.index('Path_for_mp4_video')] =  key
    df_input_output.iloc[indexi, head.index('Path_for_wav_file')] =  value
    csv_file_name = os.path.basename(value)
    csv_file_name= csv_file_name.replace(".wav",".csv")
    df_input_output.iloc[indexi, head.index('csv_file_name')] = csv_file_name
    df_input_output.iloc[indexi, head.index('csv_file_name_path_fullvideo')] = dir2 + csv_file_name
    df_input_output.iloc[indexi, head.index('csv_file_name_path_perframe')] = dir1 + csv_file_name
    #annotation_path = "/home/adrikamukherjee/data/BagOfLies/Annotations.csv"
    #df_bagoflies=pd.read_csv(annotation_path)
    if "Youtube" in value:
        if "deception" in value:
            df_input_output.iloc[indexi, head.index('label')]="Deceptive"
        if "truth" in value:
            df_input_output.iloc[indexi, head.index('label')]="Truthful"
    if "reallifedeception" in value:
        if "lie" in value:
            df_input_output.iloc[indexi, head.index('label')]="Deceptive"
        if "truth" in value:
            df_input_output.iloc[indexi, head.index('label')]="Truthful"
    if "BagOfLies" in value:
        x=os.path.split(key)[0]
        x1=os.path.split(key)[-1]
        y=os.path.split(x)[0]
        y1=os.path.split(x)[-1]
        z1=os.path.split(y)[-1]
        output_filename="./Finalised/"+z1+"/"+y1+"/"+x1
        for index, row in df_bagoflies.iterrows():
            if(row["video"]==output_filename):
                val=row["truth"]
        if(val==0):
            df_input_output.iloc[indexi, head.index('label')]="Deceptive"
        if(val==1):
            df_input_output.iloc[indexi, head.index('label')]="Truthful"
    
    
    indexi +=1  


In [6]:
print(len(df_input_output.index))
df_input_output.to_csv("Annotation_audio_features.csv",index=False) 

164


### List all .flac files of CSC Dataset and store path of these files in a list

In [17]:
csclist=[]
labellist=[]
path=r'/home/adrikamukherjee/data/CSC_Deceptice_Speech_LDC2019S09/CDC/data/'
for root, dirs, files in walk(path):
      for f in files:
        if splitext(f)[1].lower() == ".flac":
                csclist.append(join(root, f))
        if splitext(f)[1].lower() == ".ltf":
                labellist.append(join(root, f))

In [18]:
new_csc_list=[]
sublist=[]
for filename in csclist:             
    x=os.path.split(filename)[0]
    x1=os.path.split(filename)[-1]
    if "_R_" in x1:
       new_csc_list.append(filename) 
    if "_L_" in x1:
       new_csc_list.append(filename) 
    y=os.path.split(x)[0]
    y1=os.path.split(x)[-1]
    if y1 not in sublist:
        sublist.append(y1)

### Convert .flac files to .wav files

In [20]:
output_path="/home/adrikamukherjee/Data_available/CSC/"
for file in new_csc_list:
    cmd="ffmpeg -i "
    cmd = cmd + file
    x=os.path.split(file)[0]
    x1=os.path.split(file)[-1]
    op=x1.replace(".flac", ".wav")
    cmd = cmd + " "+output_path + op
    os.system(cmd)
    y=os.path.split(x)[0]
    y1=os.path.split(x)[-1]

In [6]:
output_path="/home/adrikamukherjee/Data_available/CSC/"
count_csc=0
for root, dirs, files in walk(output_path):
  for f in files:
    if splitext(f)[1].lower() == ".wav":
      count_csc +=1

In [7]:
count_csc

64

### Edit the .wav files based on labels in ".ltf" file and store the labels in another csv file

The Audio from each file is divided into sections and each section is annotated. pydub is used to divide a full audio file into segments and these segment in turn become individual datapoints.

In [10]:
from pydub import AudioSegment
from datetime import datetime

"con_ms" is used to convert time into miliseconds

In [11]:
def con_ms(v):
    i=v.find(":")
    min=v[0:i]
    #ms=int(min)*60000
    i2=v.rfind(".")
    sec=v[i+1:i2]
    ms=v[i2+1:len(v)]
    milisec=int(min)*60000+int(sec)*1000+int(ms)
    return milisec

In [12]:
con_ms("18:13.7")

1093007

### ".ltf" file from the CSC data is parsed to find the time intervals and annotations for each audio

In [21]:
input_wav_path="/home/adrikamukherjee/Data_available/CSC/"
annotation_df=pd.read_csv("/home/adrikamukherjee/Data_available/Annotation_audio_features.csv")
output_wav_path="/home/adrikamukherjee/Data_available/Audio_features/wavfiles/"
count_csc=0
c=0
for root, dirs, files in walk(input_wav_path):
  for f in files:
    file_name=input_wav_path+f
    ltf_path="/home/adrikamukherjee/data/CSC_Deceptice_Speech_LDC2019S09/CDC/data"
    i=f.find("_")
    v=f[0:i]
    ltf_path=ltf_path+"/"+v
    v=v+".ltf"
    ltf_path=ltf_path+"/"+v
    fileread = open(ltf_path, "r")
    lines = fileread.readlines()
    for val in lines:
        x = val.split()
        if x[0]=="TRUTH":
           #annotation_df.index = annotation_df.index + 1
           t1 = con_ms(x[1])
           t2 = con_ms(x[2])
           newAudio = AudioSegment.from_wav(file_name)
           newAudio = newAudio[t1:t2]
           r_f = f.replace(".wav","")
           op_f="Audio_CSC_"+r_f+"_"+str(c)
           c +=1
           csv_file_name=op_f+".csv"
           csv_file_name_path_fullvideo="/home/adrikamukherjee/Data_available/Audio_features/arff_files_full_video_cscdata/"+csv_file_name
           csv_file_name_path_perframe="/home/adrikamukherjee/Data_available/Audio_features/arff_files_frame_wise_cscdata/"+csv_file_name
           op_f=op_f+".wav"
           op_f= output_wav_path+op_f
           newAudio.export(op_f, format="wav")
           annotation_df=annotation_df.append({'label' : 'Truthful' ,'csv_file_name':csv_file_name, 'Path_for_wav_file' : op_f, 'csv_file_name_path_fullvideo': csv_file_name_path_fullvideo, 'csv_file_name_path_perframe': csv_file_name_path_perframe} , ignore_index=True)
           #annotation_df["Path_for_wav_file"]=op_f
           #annotation_df["label"]="Truthful"
        if x[0]=="LIE":
           #annotation_df.index = annotation_df.index + 1
           t1 = con_ms(x[1])
           t2 = con_ms(x[2])
           newAudio = AudioSegment.from_wav(file_name)
           newAudio = newAudio[t1:t2]
           r_f = f.replace(".wav","")
           op_f="Audio_CSC_"+r_f+"_"+str(c)
           c +=1
           csv_file_name=op_f+".csv"
           csv_file_name_path_fullvideo="/home/adrikamukherjee/Data_available/Audio_features/arff_files_full_video_cscdata/"+csv_file_name
           csv_file_name_path_perframe="/home/adrikamukherjee/Data_available/Audio_features/arff_files_frame_wise_cscdata/"+csv_file_name
           op_f=op_f+".wav"
           op_f= output_wav_path+op_f
           newAudio.export(op_f, format="wav")
           annotation_df=annotation_df.append({'label' : 'Deceptive' ,'csv_file_name':csv_file_name, 'Path_for_wav_file' : op_f, 'csv_file_name_path_fullvideo': csv_file_name_path_fullvideo, 'csv_file_name_path_perframe': csv_file_name_path_perframe} , ignore_index=True)
           #annotation_df["Path_for_wav_file"]=op_f
           #annotation_df["label"]="Deceptive"

In [22]:
annotation_df.to_csv("/home/adrikamukherjee/Data_available/Audio_features/Final_annotated_file.csv")

### Extract Features from the generated .wav using SMILExtract frame wise

In [108]:
dir = "/home/adrikamukherjee/wavfiles/"
#change outdir="/home/adrikamukherjee/Data_available/Audio_features/arff_files_frame_wise/" for other datasets
outdir="/home/adrikamukherjee/Audio_features/arff_files_frame_wise/"
list_wav=[]
wav_file_count=0
for filename in glob.glob(os.path.join(dir, '*.wav')):
    list_wav.append(filename)
    cmd=""
    input_filename=os.path.basename(filename)
    output_filename=input_filename.replace(".wav",".arff")
    cmd="./SMILExtract -C /home/adrikamukherjee/opensmile-2.3.0/config/emobase.conf -I " + "\"" + "/home/adrikamukherjee/wavfiles/" + input_filename + "\"" + " -O " + "\"" + "/home/adrikamukherjee/Audio_features/arff_files_frame_wise/" + output_filename + "\""
    cmd="cd /home/adrikamukherjee/opensmile-2.3.0; "+cmd
    x=os.system(cmd)
    if(x==0):
      wav_file_count +=1  
print(wav_file_count)

657


### Extract Features from the generated .wav using SMILExtract full video

In [110]:
dir = "/home/adrikamukherjee/wavfiles/"
#change outdir="/home/adrikamukherjee/Data_available/Audio_features/arff_files_full_video/" for other datasets
outdir="/home/adrikamukherjee/Audio_features/arff_files_full_video/"
wav_file_count=0
for filename in glob.glob(os.path.join(dir, '*.wav')):
    input_filename=os.path.basename(filename)
    output_filename=input_filename.replace(".wav",".arff")
    
    cmd="./SMILExtract -C /home/adrikamukherjee/opensmile-2.3.0/config/emobase.conf -I " + "\"" + "/home/adrikamukherjee/wavfiles/" + input_filename + "\"" + " -O " + "\"" + "/home/adrikamukherjee/Audio_features/arff_files_full_video/" + output_filename + "\""
    cmd="cd /home/adrikamukherjee/opensmile-2.3.0; "+cmd
    x=os.system(cmd)
    if(x==0):
        wav_file_count +=1
print(wav_file_count)

657


### Convert .arff to .csv

In [112]:
#dir_list=["/home/adrikamukherjee/Data_available/Audio_features/arff_files_frame_wise","/home/adrikamukherjee/Data_available/Audio_features/arff_files_full_video","/home/adrikamukherjee/Data_available/Audio_features/arff_files_full_video_cscdata","/home/adrikamukherjee/Data_available/Audio_features/arff_files_frame_wise_cscdata/"]
#dir_list=["/home/adrikamukherjee/Data_available/Audio_features/arff_files_full_video_cscdata","/home/adrikamukherjee/Data_available/Audio_features/arff_files_frame_wise_cscdata"]
#for dir in dir_list:
dir = "/home/adrikamukherjee/Data_available/Audio_features/arff_files_frame_wise"   
cmd="cd " + dir + "; python arffTocsv.py"
os.system(cmd)
    



0

### Annotate individual csv files for full video and combine them to a single file for datasets other than CSC

In [113]:
df_readannotation=pd.read_csv("/home/adrikamukherjee/deception_detection/Annotation_audio_features.csv")
df_combined=pd.DataFrame()
dir="/home/adrikamukherjee/Data_available/Audio_features/arff_files_full_video"
for filename in glob.glob(os.path.join(dir, '*.csv')):
    file=os.path.basename(filename)
    df_readindividual=pd.read_csv(filename)
    for index, row in df_readannotation.iterrows():
            if(row["csv_file_name"]==file):
                val=row["label"]
                outputpath=row["csv_file_name_path_fullvideo"]
    df_readindividual["label"]=val
    del df_readindividual['emotion']
    df_readindividual.to_csv(outputpath)
    df_combined=df_combined.append(df_readindividual,ignore_index = True)
combined_csv_path=dir+"/Combined_csv_fullvideo.csv"
df_combined.to_csv(combined_csv_path)

### Annotate individual csv files frame wise and combine them to a single file for datasets other than CSC

In [114]:
df_readannotation=pd.read_csv("/home/adrikamukherjee/deception_detection/Annotation_audio_features.csv")
df_combined=pd.DataFrame()
dir="/home/adrikamukherjee/Data_available/Audio_features/arff_files_frame_wise"
for filename in glob.glob(os.path.join(dir, '*.csv')):
    file=os.path.basename(filename)
    df_readindividual=pd.read_csv(filename)
    for index, row in df_readannotation.iterrows():
            if(row["csv_file_name"]==file):
                val=row["label"]
                outputpath=row["csv_file_name_path_perframe"]
    df_readindividual["label"]=val
    del df_readindividual['emotion']
    df_readindividual.to_csv(outputpath)
    df_combined=df_combined.append(df_readindividual,ignore_index = True)
combined_csv_path=dir+"/Combined_csv_framewise.csv"
df_combined.to_csv(combined_csv_path)

### Annotate individual csv files for full video and combine them to a single file for CSC

In [26]:
df_readannotation=pd.read_csv("/home/adrikamukherjee/Data_available/Audio_features/Final_annotated_file.csv")
df_combined=pd.DataFrame()
dir="/home/adrikamukherjee/Data_available/Audio_features/arff_files_full_video_cscdata"
for filename in glob.glob(os.path.join(dir, '*.csv')):
    file=os.path.basename(filename)
    
    df_readindividual=pd.read_csv(filename)
    for index, row in df_readannotation.iterrows():
            if(row["csv_file_name"]==file):
                val=row["label"]
                outputpath=row["csv_file_name_path_fullvideo"]

    df_readindividual["label"]=val
    del df_readindividual['emotion']
    df_readindividual.to_csv(outputpath)
    #df_combined=df_combined.append(df_readindividual,ignore_index = True)
#combined_csv_path=dir+"/Combined_csv_fullvideo_csc.csv"
#df_combined.to_csv(combined_csv_path)

In [9]:
dir="/home/adrikamukherjee/Data_available/Real-life_Deception_Detection_2016/Real-life_Deception_Detection_2016/Clips/Truthful"
for filename in glob.glob(os.path.join(dir, '*.csv')):
    os.remove(filename)

### Annotate individual csv files framewise and combine them to a single file for CSC Dataset

In [38]:
df_readannotation=pd.read_csv("/home/adrikamukherjee/Data_available/Audio_features/Final_annotated_file.csv")
df_combined=pd.DataFrame()
dir="/home/adrikamukherjee/Data_available/Audio_features/arff_files_frame_wise_cscdata3"
for filename in glob.glob(os.path.join(dir, '*.csv')):
    file=os.path.basename(filename)
    df_readindividual=pd.read_csv(filename)
    for index, row in df_readannotation.iterrows():
            if(row["csv_file_name"]==file):
                val=row["label"]
                outputpath=row["csv_file_name_path_perframe"]
    df_readindividual["label"]=val
    del df_readindividual['emotion']
    outputpath=outputpath.replace("arff_files_frame_wise_cscdata","arff_files_frame_wise_cscdata3")
    df_readindividual.to_csv(outputpath)
    #df_combined=df_combined.append(df_readindividual,ignore_index = True)
#combined_csv_path=dir+"/Combined_csv_framewise_csc.csv"
#df_combined.to_csv(combined_csv_path)

### Copy all .csv files of CSC and non-CSC Dataset for full_video to a new folder

In [32]:
list_dir=["/home/adrikamukherjee/Data_available/Audio_features/arff_files_full_video","/home/adrikamukherjee/Data_available/Audio_features/arff_files_full_video_cscdata"]
out_dir="/home/adrikamukherjee/Data_available/Audio_features/csv_full_audio"
for dir in list_dir:
    for filename in glob.glob(os.path.join(dir, '*.csv')):
            file=os.path.basename(filename)
            out_path=out_dir+"/"+file
            cmd="cp "+filename+" "+out_path
            os.system(cmd)
    
    

### Copy all .csv files of CSC and non-CSC Dataset for frame_wise to a new folder

In [33]:
list_dir=["/home/adrikamukherjee/Data_available/Audio_features/arff_files_frame_wise","/home/adrikamukherjee/Data_available/Audio_features/arff_files_frame_wise_cscdata"]
out_dir="/home/adrikamukherjee/Data_available/Audio_features/csv_framewise_audio"
for dir in list_dir:
    for filename in glob.glob(os.path.join(dir, '*.csv')):
            file=os.path.basename(filename)
            out_path=out_dir+"/"+file
            cmd="cp "+filename+" "+out_path
            os.system(cmd)

### Combine all data for framewise video

In [ ]:
input_dir1="/home/adrikamukherjee/Data_available/Audio_features/csv_framewise_audio"
df_frame=pd.DataFrame()
for filename in glob.glob(os.path.join(input_dir1, '*.csv')):
        file=os.path.basename(filename)
        if (file=="Combined_csv_framewise_csc.csv" or file == "Combined_csv_framewise.csv"):
            df=pd.read_csv(filename)
            df_frame=df_frame.append(df)       
df_frame.to_csv("/home/adrikamukherjee/Data_available/Audio_features/csv_framewise_audio/Final_dataset_Audio_framewise.csv")


### Combine all data for full video

In [ ]:
input_dir2="/home/adrikamukherjee/Data_available/Audio_features/csv_full_audio"
df_full=pd.DataFrame()
for filename in glob.glob(os.path.join(input_dir1, '*.csv')):
        file=os.path.basename(filename)
        if (file=="Combined_csv_fullvideo_csv.csv" or file == "Combined_csv_fullvideo.csv"):
            df=pd.read_csv(filename)
            df_full=df_full.append(df)
df_full.to_csv("/home/adrikamukherjee/Data_available/Audio_features/csv_full_audio/Final_dataset_Audio_fullvideo.csv")